# Yelp ADI

- Robert Yonce
- 5/3/23

# Imports

In [4]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Keys & Login

In [5]:
# Keys

with open('/Users/robertyonce/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [6]:
# YelpAPI

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api


# Set Parameters and File

In [7]:
# set our API call parameters 
LOCATION = 'Knoxville,TN'
TERM = 'Asian'

In [8]:
# Specifying JSON_FILE filename and including the search terms in the filename

JSON_FILE_KNOX = "Data/results_in_progress_Knox_Asian.json"
JSON_FILE_KNOX

'Data/results_in_progress_Knox_Asian.json'

In [9]:
# Check if JSON_FILE_KNOX exists

file_exists = os.path.isfile(JSON_FILE_KNOX)

# If it does not exist: 

if file_exists == False:
    
    # Create New Folder if needed
    # Get the Folder Name only
    
    folder = os.path.dirname(JSON_FILE_KNOX)
    
    # If JSON_FILE_KNOX included a folder:
    
    if len(folder)>0:
        
        # create the folder
        
        os.makedirs(folder,exist_ok=True)
        
        
    # INFORM USER AND SAVE EMPTY LIST
    
    print(f'[i] {JSON_FILE_KNOX} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE_KNOX,'w') as f:
        json.dump([],f)  
        
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE_KNOX} already exists.")

[i] Data/results_in_progress_Knox_Asian.json not found. Saving empty list to file.


In [10]:
# Load previous results and use len of results for offset

with open(JSON_FILE_KNOX,'r') as f:
    previous_results = json.load(f)
    
# Set offset based on previous results

n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


# API CALL

In [11]:
# use our yelp_api variable's search_query method to perform our API call

results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
# How many results total?

total_results = results['total']
total_results


258

In [13]:
# Results per page

results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
# Use math.ceil to round up for the total number of pages of results.

n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

13

# Extend and Loop

In [15]:
# Join new results with old list with extend and save to file

previous_results.extend(results['businesses'])  
with open(JSON_FILE_KNOX,'w') as f:
     json.dump(previous_results,f)

In [16]:
# Loop through pages showing time

for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/13 [00:00<?, ?it/s]

# Final DF

In [17]:
# Load final results

final_df = pd.read_json(JSON_FILE_KNOX)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-sjr3fdVEisP9Y5Z3MLnxQ,kaizen-knoxville-2,Kaizen,https://s3-media4.fl.yelpcdn.com/bphoto/hKHnwn...,False,https://www.yelp.com/biz/kaizen-knoxville-2?ad...,289,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 35.9694209, 'longitude': -83.918324}",[delivery],$$,"{'address1': '127 S Central Ave', 'address2': ...",18654094444,(865) 409-4444,1460.420272
1,1kqFUhwnvJ0k-BA3Ca7tHQ,gogi-korean-kitchen-knoxville,Gogi Korean Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/6I52B0...,False,https://www.yelp.com/biz/gogi-korean-kitchen-k...,30,"[{'alias': 'korean', 'title': 'Korean'}]",4.5,"{'latitude': 35.92508469720469, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '7818 Montvue Center Way', 'addre...",18652815797,(865) 281-5797,11229.680535
2,Lrz9QwH4HK8EDTkcZ8kBUg,szechuan-garden-chinese-restaurant-knoxville,Szechuan Garden Chinese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/w1jq9k...,False,https://www.yelp.com/biz/szechuan-garden-chine...,106,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 35.93579709084952, 'longitude': -...",[delivery],$,"{'address1': '4211 Chapman Hwy', 'address2': '...",18655790889,(865) 579-0889,2862.519676
3,vSUM4N3XKa11wGltYSMytw,sticky-rice-cafe-knoxville,Sticky Rice Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/Ldiod-...,False,https://www.yelp.com/biz/sticky-rice-cafe-knox...,609,"[{'alias': 'laotian', 'title': 'Laotian'}, {'a...",4.5,"{'latitude': 35.9266842496109, 'longitude': -8...",[delivery],$$,"{'address1': '120 Jack Dance St', 'address2': ...",18652496273,(865) 249-6273,11236.843006
4,YR29u_bjgrKELlfVpZO8yQ,seoul-brothers-knoxville,Seoul Brothers,https://s3-media3.fl.yelpcdn.com/bphoto/HuIANl...,False,https://www.yelp.com/biz/seoul-brothers-knoxvi...,26,"[{'alias': 'korean', 'title': 'Korean'}]",4.5,"{'latitude': 35.97026, 'longitude': -83.92195}",[],NaN,"{'address1': '333 W Depot Ave', 'address2': ''...",18659738779,(865) 973-8779,1467.360708


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
15,wqLSTPlop9tQ5O0JH7kzWw,chaiyo-thai-and-sushi-bar-knoxville,Chaiyo Thai & Sushi Bar,https://s3-media2.fl.yelpcdn.com/bphoto/emUAsr...,False,https://www.yelp.com/biz/chaiyo-thai-and-sushi...,140,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.0,"{'latitude': 35.959541, 'longitude': -83.92935...","[delivery, pickup]",$$,"{'address1': '601 James Agee St', 'address2': ...",18652498081,(865) 249-8081,498.375071
16,H4EW_19k_xX8OjcEEGdb9A,jai-dee-thai-kitchen-knoxville,Jai Dee Thai Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/Nwh387...,False,https://www.yelp.com/biz/jai-dee-thai-kitchen-...,105,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",4.5,"{'latitude': 35.9561768, 'longitude': -83.9323...","[delivery, pickup]",$$,"{'address1': '823 Melrose Pl', 'address2': '',...",18652497539,(865) 249-7539,695.925662
17,6-mv5IFsVOOL6aab1a4HpA,thai-time-knoxville,Thai Time,https://s3-media2.fl.yelpcdn.com/bphoto/oXceyx...,False,https://www.yelp.com/biz/thai-time-knoxville?a...,4,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",5.0,"{'latitude': 35.921542, 'longitude': -83.874781}","[delivery, pickup]",NaN,"{'address1': '6200 Chapman Hwy', 'address2': '...",18652537185,(865) 253-7185,5989.004556
18,ARMhMfwNWu4F78J9xJQ8Bg,anaba-japanese-cuisine-northshore-knoxville,Anaba Japanese Cuisine-Northshore,https://s3-media3.fl.yelpcdn.com/bphoto/KUrpMi...,False,https://www.yelp.com/biz/anaba-japanese-cuisin...,189,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 35.8641, 'longitude': -84.06415}",[delivery],$$,"{'address1': '9405 S Northshore Dr', 'address2...",18655312363,(865) 531-2363,16278.706158
19,55tkqqsi2hmd_CPnMXb2XQ,ichiban-asian-cuisine-knoxville,Ichiban Asian Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/ygVC6P...,False,https://www.yelp.com/biz/ichiban-asian-cuisine...,87,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 35.9054607332768, 'longitude': -8...","[delivery, pickup]",$$,"{'address1': '7507 S Northshore Dr', 'address2...",18656907999,(865) 690-7999,10319.746911
